In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definer variabler der bruges til at filtrere data med.

In [2]:
NB = pd.to_datetime('01-01-2023') #indflytningsdato for NordBlok NB

MA = 10 #Maksimal alder i år ved indflytningen

NBMA = pd.to_datetime('01-01-' + str(NB.year - MA)) #dato for hvornår udstyr der kan flyttes i NordBlokken
#NBMA = NB.year - MA 

BG = 5000 #BagatelGrænse for pris

EJER = 'Bispebjerg og Frederiksberg Hospital'
#EJER = 'Nordsjællands Hospital'
#print(NBMA)
FilterInput = pd.DataFrame({'Filter funktion':['Ejer', 'Indflytnings dato', 'Maksimal alder [år]','Maksimal alder [dato]', 'Bagatelgrænse i kr', 'Ekskluderede grupper'],
                           'Værdi': [EJER, NB, MA, NBMA, BG, 'nan']})

# Læs data

In [3]:
filename = 'BBH+FRH 19-03-20.xlsx' #'BBH+FRH 01-07-19.xlsx'
#filename = 'myxls.xlsx'

df = pd.read_excel(filename, sheet_name='Data') # Medusa udtræk

gz = pd.read_excel(filename, sheet_name='Gråzone') # Gråzone udstyr

xmd = pd.read_excel(filename, sheet_name='Xmodel') # Ekskluderete modeller

In [8]:
#klargør plot af forskellige ting
def opgorelse(df, graf=False, xls = False, filNavn ='AntalGrupper.xlsx'):
    data = df['Gruppenavn']
    data.fillna('#fejl#')
   
    antal = df['Gruppenavn'].value_counts().to_frame()
    antal.reset_index(inplace = True)
    antal.rename(columns={"index": "Gruppe", "Gruppenavn": "Antal"}, inplace = True)
   
    if graf:
        #%matplotlib notebook
        d = antal.loc[antal['Antal'] > 50]
        gr = d['Gruppe']
        an = d['Antal']
        plt.rcdefaults()
        fig, ax = plt.subplots()
        ax.barh(gr,an)
        ax.invert_yaxis()
        plt.show()
    
    if xls:
        with pd.ExcelWriter(filNavn) as writer:
            antal.to_excel(writer,index = False)
    
    return antal

d = opgorelse(df, xls =True)
d.describe()

,Antal
count,488.000000
mean,19.272541
std,60.982220
min,1.000000
25%,1.000000
50%,3.000000
75%,11.000000
max,710.000000


In [5]:
#Tilpasninger
df['Indkøbt']= pd.to_datetime(df.Indkøbt)#konverter til datetime som Pandas kan regne på

df["Pris"] = df[["Indkøbspris","Genansk.pris"]].max(axis=1)# find den største værdi og sæt det ind i ny kollonde "Pris"
#df.head()

# Så begynder filtereringen

In [6]:
x = df['Ejer (overafdeling)'].str.split(";", n = 1, expand = True)
df["Overafdeling"] =x[0]
df["Ejer"] = x[1]

In [7]:
fdf = df.loc[df.Ejer.str.contains(EJER)== True] #filterer på Ejer
#fdf.head()

In [8]:
fdf = fdf.loc[fdf.Indkøbt > NBMA,:] #filtrer på alder
#fdf.head()

In [9]:
fdf =fdf.loc[fdf.Pris > BG,:] #filtrer på pris
#fdf.head()

In [10]:
fdf = fdf.loc[~fdf.Gruppenavn.isin(gz.Gruppenavn)] #fjern gråzone udstyr

In [11]:
#fdf = fdf.loc[~fdf.Model.isin(xmd.Model),:] # fjern ekskluderede modeller

In [12]:
fdf = fdf.drop(['Indkøbspris', 'Genansk.pris', 'Ejer (overafdeling)'], axis=1)

In [13]:
cols = fdf.columns.tolist()
#print(cols)

In [14]:
cols = ['App.nr', 'Gruppenavn', 'Fabrikat', 'Model', 'Serie nr.', 'Indkøbt', 'Pris', 'Overafdeling', 'Afdeling (placering)', 'Afsnit (Placering)', 'Ejer']
fdf = fdf[cols]

In [15]:
#fdf.info()

# Opdeling i afdelinger

In [20]:
Afd = pd.unique(fdf['Overafdeling'])
Afd.sort()

Afd = np.append(Afd,'Total')
#print(Afd)

fdf['Overafdeling'] = fdf['Overafdeling'].str.slice(0,27)
afd = pd.unique(fdf['Overafdeling'])
afd.sort()

i=0
antal =[]
for x in afd:
    xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
    antal.append(len(xls))
    i = i+len(xls)
   # print(x[0:28], "  ",len(xls))    
#print(i)
antal.append(i)

StatusArk =pd.DataFrame(list(zip(Afd, antal)), columns =['Afdeling', 'Antal udstyr'])
#StatusArk

['AMA., Akut modtageafdeling ' 'Abdominalcenter K' 'Anæstesi'
 'Audiologisk ambulatorie, ud' 'Center for Klinisk Forsknin'
 'Centraloperationsgang, FRH' 'Dermato-Venerologisk Afd. +'
 'Endokrinologisk Overafdelin' 'Fysio- og ergoterapiafdelin'
 'Fælles for BBH og FRH Hospi' 'Geriatrisk afdeling G'
 'Geriatrisk og Palliativ ove' 'IMT-afdeling' 'Institut for idrætsmedicin '
 'KVU - Kvalitet og Uddannels' 'Kardiologisk afdeling Y'
 'Klinisk Farmakologisk Afdel' 'Klinisk Fysiologisk Afdelin'
 'Klinisk biokemisk afdeling' 'Koordinerende Forskningsenh'
 'Lungemedicinsk afdeling L' 'Medicinsk og Geriatrisk Ove'
 'Neurologisk afd. N' 'Ortopædkirurgisk afd. M'
 'Ortopædkirurgisk afdeling O' 'Parkerinstituttet'
 'Reumatologisk afdeling H, F' 'Rigshospitalets Udefunktion'
 'Røntgenafdeling' 'Tværsektoriel Forskningsenh' 'Udviklingsafdelingen'
 'Urologisk afdeling U, FRH' 'Total']


In [21]:
with pd.ExcelWriter('Filtreret_og_Opdelt.xlsx') as writer:
    FilterInput.to_excel(writer, sheet_name='Opsumering',index = False)
    gz.to_excel(writer, sheet_name='Opsumering',startrow = 6,startcol = 1,index = False)
    StatusArk.to_excel(writer, sheet_name='Opsumering',startcol = 4, index = False)
    
    for x in afd:
        xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
        if len(xls)>0:
            xls.to_excel(writer, sheet_name=x[0:27],index = False)

In [18]:
#d = opgorelse(fdf, True)
#d

## Skriv det hele i et regneark

In [25]:
with pd.ExcelWriter('Filtreret.xlsx') as writer:
    
    fdf.to_excel(writer,index = False)